## Eric Johnson, Video Game Music Composers

The general process is outlined here, but there are a couple of things I need to do. 

First and foremost, I'm going to go back and check my data cleaning. I've already done a considerable amount using OpenRefine in the last few days to get more games/soundtracks to match up, but I'm going to see what else I can do. 

Also on that note, I'm going to go through and physically look and check to see if I should expect a lot of these games to line up when combining my final dataset. I expected that they should, but once I have put all the data together, when I do all inner joins, I'm only getting a handful of unique rows (after starting with around 16,000). Needless to say, I was initially expecting a final number a bit higher. 

### Update
I might need to get rid of all the vgmdb datasets (this would still leave me with 4 datasets total) because without them, I get 535 unique rows of data using inner joins (compared to 43 using the vgmdb data). This is actually not that bad, considering my smallest dataset only has 983 rows. So that's 535/983 or 54.43% matching rows to all other 3 datasets. Regardless, I'll take a closer look into possible issues with the vgmdb data and see if the issue is data cleaning, or if the dataset is just overall more unhelpful than I thought it was going to be based on the other data I've collected. 

#### Final Steps

After all of that, I'll need to just clean up the notebook and add better narrative and comments. 

In [237]:
import pandas as pd

First, all of the datasets need to be read in.

In [238]:
kaggle_file = '../Datasets/Kaggle/video-game-sales-with-ratings/video_game_sales_cleaned.csv'
kaggle = pd.read_csv(kaggle_file)

In [239]:
wiki_file = '../Datasets/Wikipedia/List of Video Game Musicians/all_composer_data_cleaned_adjusted.txt'
wiki = pd.read_csv(wiki_file, sep=';')

In [240]:
vgmdb_artist_file = '../Datasets/vgmdb/highest_rated_artists_cleaned_adjusted.txt'
vgmdb_artist = pd.read_csv(vgmdb_artist_file, sep='<')

In [241]:
# vgmdb_most_pop_album_file = '../Datasets/vgmdb/most_popular_albums_adjusted.txt'
# vgmdb_most_pop_album = pd.read_csv(vgmdb_most_pop_album_file)

In [242]:
vgmdb_least_pop_album_file = '../Datasets/vgmdb/least_popular_albums_cleaned_adjusted.txt'
vgmdb_least_pop_album = pd.read_csv(vgmdb_least_pop_album_file)

In [243]:
mp3_downloads_file = '../Datasets/Video Game Music mp3 Downloads/top_1000_downloaded_soundtracks_cleaned_adjusted.txt'
mp3_downloads = pd.read_csv(mp3_downloads_file, sep=';')

Taking a look at the Kaggle dataset, to make sure everything looks okay:

In [244]:
kaggle.head(20)

,Name,Platform,Year_of_Release,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales,Critic_Score,Critic_Count,User_Score,User_Count,Developer,Rating,Column 17,Column 18,Column 19
0,Wii Sports,Wii,2006,Sports,Nintendo,41.36,28.96,3.77,8.45,82.53,76.0,51.0,8,322,Nintendo,E,NaN,NaN,NaN
1,Super Mario Bros.,NES,1985,Platform,Nintendo,29.08,3.58,6.81,0.77,40.24,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Mario Kart Wii,Wii,2008,Racing,Nintendo,15.68,12.76,3.79,3.29,35.52,82.0,73.0,8.3,709,Nintendo,E,NaN,NaN,NaN
3,Wii Sports Resort,Wii,2009,Sports,Nintendo,15.61,10.93,3.28,2.95,32.77,80.0,73.0,8,192,Nintendo,E,NaN,NaN,NaN
4,Pokemon Red/green/blue/yellow,GB,1996,Role-Playing,Nintendo,11.27,8.89,10.22,1.00,31.37,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,Tetris,GB,1989,Puzzle,Nintendo,23.2,2.26,4.22,0.58,30.26,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,New Super Mario Bros.,DS,2006,Platform,Nintendo,11.28,9.14,6.5,2.88,29.80,89.0,65.0,8.5,431,Nintendo,E,NaN,NaN,NaN
7,Wii Play,Wii,2006,Misc,Nintendo,13.96,9.18,2.93,2.84,28.92,58.0,41.0,6.6,129,Nintendo,E,NaN,NaN,NaN
8,New Super Mario Bros. Wii,Wii,2009,Platform,Nintendo,14.44,6.94,4.7,2.24,28.32,87.0,80.0,8.4,594,Nintendo,E,NaN,NaN,NaN
9,Duck Hunt,NES,1984,Shooter,Nintendo,26.93,0.63,0.28,0.47,28.31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


We can now extract just the columns that we want from this dataset. 

In [245]:
kaggle = kaggle.loc[:, ['Name', 'Platform', 'Year_of_Release', 'Genre', 'Publisher', 'Critic_Score', 'User_Score', 
                        'Developer', 'Rating']]
kaggle.head(20)

,Name,Platform,Year_of_Release,Genre,Publisher,Critic_Score,User_Score,Developer,Rating
0,Wii Sports,Wii,2006,Sports,Nintendo,76.0,8,Nintendo,E
1,Super Mario Bros.,NES,1985,Platform,Nintendo,NaN,NaN,NaN,NaN
2,Mario Kart Wii,Wii,2008,Racing,Nintendo,82.0,8.3,Nintendo,E
3,Wii Sports Resort,Wii,2009,Sports,Nintendo,80.0,8,Nintendo,E
4,Pokemon Red/green/blue/yellow,GB,1996,Role-Playing,Nintendo,NaN,NaN,NaN,NaN
5,Tetris,GB,1989,Puzzle,Nintendo,NaN,NaN,NaN,NaN
6,New Super Mario Bros.,DS,2006,Platform,Nintendo,89.0,8.5,Nintendo,E
7,Wii Play,Wii,2006,Misc,Nintendo,58.0,6.6,Nintendo,E
8,New Super Mario Bros. Wii,Wii,2009,Platform,Nintendo,87.0,8.4,Nintendo,E
9,Duck Hunt,NES,1984,Shooter,Nintendo,NaN,NaN,NaN,NaN


Next, let's take a look at the Wikipedia dataset to make sure it looks okay. 

In [246]:
wiki.head(20)

,Soundtrack,Composer,Composer Birthday
0,1942,Akari Kaida,1974-01-10
1,Ace Attorney,Akari Kaida,1974-01-10
2,Bionic Commando,Akari Kaida,1974-01-10
3,Breath Of Fire,Akari Kaida,1974-01-10
4,Buster Bros.,Akari Kaida,1974-01-10
5,Commando,Akari Kaida,1974-01-10
6,Darkstalkers,Akari Kaida,1974-01-10
7,Dark Void,Akari Kaida,1974-01-10
8,Dead Rising,Akari Kaida,1974-01-10
9,Devil May Cry,Akari Kaida,1974-01-10


That looks okay, so we'll check the Highest Rated Artist dataset from vgmdb.net, as that is the first one we will combine with the Wikipedia dataset. 

In [247]:
vgmdb_artist.head(20)

,Rank,Artist Name,Rating,#votes,Column 4
0,1,Hiromi Uehara,4.98,22.0,NaN
1,2,Kyle Scott,4.98,21.0,NaN
2,3,Kou Nakamura,4.94,43.0,NaN
3,4,Janne Sala,4.94,24.0,NaN
4,5,Alessandro Salerno,4.9,21.0,NaN
5,6,Asuka Oda,4.9,68.0,NaN
6,7,Junpei Ohno,4.9,25.0,NaN
7,8,Meiko Nakahara,4.89,31.0,NaN
8,9,Monami Okawa,4.88,65.0,NaN
9,10,Miho Matsumoto,4.88,21.0,NaN


In [248]:
vgmdb_artist = vgmdb_artist.loc[:, ['Rank', 'Artist Name', 'Rating']]
vgmdb_artist.head(20)

,Rank,Artist Name,Rating
0,1,Hiromi Uehara,4.98
1,2,Kyle Scott,4.98
2,3,Kou Nakamura,4.94
3,4,Janne Sala,4.94
4,5,Alessandro Salerno,4.9
5,6,Asuka Oda,4.9
6,7,Junpei Ohno,4.9
7,8,Meiko Nakahara,4.89
8,9,Monami Okawa,4.88
9,10,Miho Matsumoto,4.88


Looking at the column names for each of these datasets, there seems to be some extra spaces that we could remove:

In [249]:
vgmdb_artist.dtypes

Rank            int64
Artist Name    object
Rating         object
dtype: object

In [250]:
wiki.dtypes

Soundtrack           object
Composer             object
Composer Birthday    object
dtype: object

We can easily remove these extra white spaces at the beginning of the column names using a basic lamdba function. This function was provided with assistance from https://stackoverflow.com/questions/21606987/how-can-i-strip-the-whitespace-from-pandas-dataframe-headers. 

In [251]:
wiki = wiki.rename(columns=lambda x: x.strip())
wiki.columns

Index(['Soundtrack', 'Composer', 'Composer Birthday'], dtype='object')

In [252]:
vgmdb_artist = vgmdb_artist.rename(columns=lambda x: x.strip())
vgmdb_artist.columns

Index(['Rank', 'Artist Name', 'Rating'], dtype='object')

In [253]:
vgmdb_artist = vgmdb_artist.rename(columns={'Artist Name': 'Composer'})
vgmdb_artist.head(20)

,Rank,Composer,Rating
0,1,Hiromi Uehara,4.98
1,2,Kyle Scott,4.98
2,3,Kou Nakamura,4.94
3,4,Janne Sala,4.94
4,5,Alessandro Salerno,4.9
5,6,Asuka Oda,4.9
6,7,Junpei Ohno,4.9
7,8,Meiko Nakahara,4.89
8,9,Monami Okawa,4.88
9,10,Miho Matsumoto,4.88


In [254]:
wiki['Composer'].head(20)

0     Akari Kaida
1     Akari Kaida
2     Akari Kaida
3     Akari Kaida
4     Akari Kaida
5     Akari Kaida
6     Akari Kaida
7     Akari Kaida
8     Akari Kaida
9     Akari Kaida
10    Akari Kaida
11    Akari Kaida
12    Akari Kaida
13    Akari Kaida
14    Akari Kaida
15    Akari Kaida
16    Akari Kaida
17    Akari Kaida
18    Akari Kaida
19    Akari Kaida
Name: Composer, dtype: object

In [255]:
# If I keep the below as an inner join, there are 5384 rows, compared to 18,000-something if I use an outer join

We can now combine these two datasets on the Composer/Artist Name columns. For now, I'm using an inner join, but this might change once I take a closer look at all the data (switch to an outer join to include all data possibly, even missing data, but we'll see)

TRYING OUTER NOW FOR THIS ONE TO GET ALL ARTIST NAMES POSSIBLE

In [256]:
vgmdb_artist_merge_wiki = pd.merge(wiki, vgmdb_artist, how='outer', on='Composer')
vgmdb_artist_merge_wiki

,Soundtrack,Composer,Composer Birthday,Rank,Rating
0,1942,Akari Kaida,1974-01-10,7252.0,4.04
1,Ace Attorney,Akari Kaida,1974-01-10,7252.0,4.04
2,Bionic Commando,Akari Kaida,1974-01-10,7252.0,4.04
3,Breath Of Fire,Akari Kaida,1974-01-10,7252.0,4.04
4,Buster Bros.,Akari Kaida,1974-01-10,7252.0,4.04
5,Commando,Akari Kaida,1974-01-10,7252.0,4.04
6,Darkstalkers,Akari Kaida,1974-01-10,7252.0,4.04
7,Dark Void,Akari Kaida,1974-01-10,7252.0,4.04
8,Dead Rising,Akari Kaida,1974-01-10,7252.0,4.04
9,Devil May Cry,Akari Kaida,1974-01-10,7252.0,4.04


In [257]:
kaggle = kaggle.rename(columns={'Name': 'Game', 'Rating': 'ESRB Rating'})
kaggle.head(20)

,Game,Platform,Year_of_Release,Genre,Publisher,Critic_Score,User_Score,Developer,ESRB Rating
0,Wii Sports,Wii,2006,Sports,Nintendo,76.0,8,Nintendo,E
1,Super Mario Bros.,NES,1985,Platform,Nintendo,NaN,NaN,NaN,NaN
2,Mario Kart Wii,Wii,2008,Racing,Nintendo,82.0,8.3,Nintendo,E
3,Wii Sports Resort,Wii,2009,Sports,Nintendo,80.0,8,Nintendo,E
4,Pokemon Red/green/blue/yellow,GB,1996,Role-Playing,Nintendo,NaN,NaN,NaN,NaN
5,Tetris,GB,1989,Puzzle,Nintendo,NaN,NaN,NaN,NaN
6,New Super Mario Bros.,DS,2006,Platform,Nintendo,89.0,8.5,Nintendo,E
7,Wii Play,Wii,2006,Misc,Nintendo,58.0,6.6,Nintendo,E
8,New Super Mario Bros. Wii,Wii,2009,Platform,Nintendo,87.0,8.4,Nintendo,E
9,Duck Hunt,NES,1984,Shooter,Nintendo,NaN,NaN,NaN,NaN


In [258]:
vgmdb_artist_merge_wiki = vgmdb_artist_merge_wiki.rename(
    columns={'Soundtrack': 'Game', 'Rank': 'Composer Rank', 
             'Rating': 'Composer Rating'})
vgmdb_artist_merge_wiki.head(20)

,Game,Composer,Composer Birthday,Composer Rank,Composer Rating
0,1942,Akari Kaida,1974-01-10,7252.0,4.04
1,Ace Attorney,Akari Kaida,1974-01-10,7252.0,4.04
2,Bionic Commando,Akari Kaida,1974-01-10,7252.0,4.04
3,Breath Of Fire,Akari Kaida,1974-01-10,7252.0,4.04
4,Buster Bros.,Akari Kaida,1974-01-10,7252.0,4.04
5,Commando,Akari Kaida,1974-01-10,7252.0,4.04
6,Darkstalkers,Akari Kaida,1974-01-10,7252.0,4.04
7,Dark Void,Akari Kaida,1974-01-10,7252.0,4.04
8,Dead Rising,Akari Kaida,1974-01-10,7252.0,4.04
9,Devil May Cry,Akari Kaida,1974-01-10,7252.0,4.04


This combined dataset can now be joined with the Kaggle dataset on the Soundtrack and Game Name columns. Again, I'm just using an inner join for now to exclude a lot of incomplete data, but might change this to outer join in the final version. 

INNER JOIN FOR THIS ONE 

In [259]:
vgmdb_wiki_kaggle = pd.merge(kaggle, vgmdb_artist_merge_wiki, on='Game')
vgmdb_wiki_kaggle

,Game,Platform,Year_of_Release,Genre,Publisher,Critic_Score,User_Score,Developer,ESRB Rating,Composer,Composer Birthday,Composer Rank,Composer Rating
0,Wii Sports,Wii,2006,Sports,Nintendo,76.0,8,Nintendo,E,Kazumi Totaka,1967-08-23,6777.0,4.07
1,Wii Sports,Wii,2006,Sports,Nintendo,76.0,8,Nintendo,E,Kazumi Totaka,1967-08-23,6777.0,4.07
2,Pokemon Red/green/blue/yellow,GB,1996,Role-Playing,Nintendo,NaN,NaN,NaN,NaN,Junichi Masuda,1968-01-12,5712.0,4.14
3,Tetris,GB,1989,Puzzle,Nintendo,NaN,NaN,NaN,NaN,Hirokazu Tanaka,1957-12-13,6489.0,4.09
4,Tetris,GB,1989,Puzzle,Nintendo,NaN,NaN,NaN,NaN,Jun Senoue,1970-08-02,6091.0,4.12
5,Tetris,GB,1989,Puzzle,Nintendo,NaN,NaN,NaN,NaN,Kazumi Totaka,1967-08-23,6777.0,4.07
6,Tetris,GB,1989,Puzzle,Nintendo,NaN,NaN,NaN,NaN,Tomoya Ohtani,1974-07-01,5079.0,4.18
7,Tetris,NES,1988,Puzzle,Nintendo,NaN,NaN,NaN,NaN,Hirokazu Tanaka,1957-12-13,6489.0,4.09
8,Tetris,NES,1988,Puzzle,Nintendo,NaN,NaN,NaN,NaN,Jun Senoue,1970-08-02,6091.0,4.12
9,Tetris,NES,1988,Puzzle,Nintendo,NaN,NaN,NaN,NaN,Kazumi Totaka,1967-08-23,6777.0,4.07


Some of the rows seem to be exact duplicates of each other, so we can drop any duplicate records. 

In [260]:
vgmdb_wiki_kaggle.drop_duplicates()

,Game,Platform,Year_of_Release,Genre,Publisher,Critic_Score,User_Score,Developer,ESRB Rating,Composer,Composer Birthday,Composer Rank,Composer Rating
0,Wii Sports,Wii,2006,Sports,Nintendo,76.0,8,Nintendo,E,Kazumi Totaka,1967-08-23,6777.0,4.07
2,Pokemon Red/green/blue/yellow,GB,1996,Role-Playing,Nintendo,NaN,NaN,NaN,NaN,Junichi Masuda,1968-01-12,5712.0,4.14
3,Tetris,GB,1989,Puzzle,Nintendo,NaN,NaN,NaN,NaN,Hirokazu Tanaka,1957-12-13,6489.0,4.09
4,Tetris,GB,1989,Puzzle,Nintendo,NaN,NaN,NaN,NaN,Jun Senoue,1970-08-02,6091.0,4.12
5,Tetris,GB,1989,Puzzle,Nintendo,NaN,NaN,NaN,NaN,Kazumi Totaka,1967-08-23,6777.0,4.07
6,Tetris,GB,1989,Puzzle,Nintendo,NaN,NaN,NaN,NaN,Tomoya Ohtani,1974-07-01,5079.0,4.18
7,Tetris,NES,1988,Puzzle,Nintendo,NaN,NaN,NaN,NaN,Hirokazu Tanaka,1957-12-13,6489.0,4.09
8,Tetris,NES,1988,Puzzle,Nintendo,NaN,NaN,NaN,NaN,Jun Senoue,1970-08-02,6091.0,4.12
9,Tetris,NES,1988,Puzzle,Nintendo,NaN,NaN,NaN,NaN,Kazumi Totaka,1967-08-23,6777.0,4.07
10,Tetris,NES,1988,Puzzle,Nintendo,NaN,NaN,NaN,NaN,Tomoya Ohtani,1974-07-01,5079.0,4.18


Next we can look at the Most and Least Popular Albums data from vgmdb.net.

In [261]:
# vgmdb_most_pop_album

In [262]:
vgmdb_least_pop_album.head(20)

,Rank,Album Name,Rating,#votes,Popularity
0,1,Diabolik Lovers,3.80,15,0.0006
1,2,Eiga Precure Dream Stars!,4.10,15,0.0006
2,3,Diabolik Lovers,4.04,14,0.0006
3,4,Haikyu,4.04,14,0.0006
4,5,Eiga Kirakira precure a La Mode Paris-tto! Omo...,4.50,13,0.0006
5,6,Nightmare,4.31,13,0.0006
6,14,Dive & Drama,5.00,11,0.0006
7,16,Square Enix,4.58,13,0.0029
8,18,Kirakira precure a La Mode 1: Precure sound de...,4.33,15,0.0040
9,19,Ao Haru Ride,4.29,14,0.0043


These two datasets are *almost* reverse copies of each other, but there are a couple of entries (at least looking at the least popular in both sets) that slightly differ. So we can combine these two datasets into one to account for all information, rather than just discarding one. 

First we'll strip the white spaces from the header names like before. 

In [263]:
# vgmdb_least_pop_album = vgmdb_least_pop_album.rename(columns=lambda x: x.strip())
# vgmdb_least_pop_album.columns

The data that differs between these two datasets is the Rank. So we should rename this column in each dataset to reflect where it came from. Here, we'll rename it to Least Popularity Rank. 

In [264]:
# vgmdb_least_pop_album = vgmdb_least_pop_album.rename(columns={'Rank': 'Least Popularity Rank'})
# vgmdb_least_pop_album

And strip the white space and rename Rank to Most Popularity Rank in the other dataset:

In [265]:
# vgmdb_most_pop_album = vgmdb_most_pop_album.rename(columns=lambda x: x.strip())
# vgmdb_most_pop_album.columns

In [266]:
# vgmdb_most_pop_album = vgmdb_most_pop_album.rename(columns={'Rank': 'Most Popularity Rank'})
# vgmdb_most_pop_album

Now, we're ready to merge these two datasets into one. There are a number of columns that overlap, so we'll be combining them on the Album Name, Rating, #votes, and Popularity columns. I might change this to just combine on the Album Name column, as it should produce the exact same results. 

In [267]:
# vgmdb_all_popularity = pd.merge(vgmdb_most_pop_album, vgmdb_least_pop_album, on=['Album Name', 'Rating', '#votes', 
#                                                                                  'Popularity'])
# vgmdb_all_popularity

Now that that dataset is ready, let's look at the mp3 Downloads dataset. This is the next set that we will combine with the vgmdb datasets we just worked on. 

In [268]:
mp3_downloads.head(20)

,#,Album
0,1,Persona 5
1,2,Need For Speed: Most Wanted
2,3,Super Mario World
3,4,Minecraft
4,5,Legend Of Zelda: Ocarina Of Time
5,6,Super Smash Bros Brawl: Gamerip
6,7,Persona 4
7,8,Need For Speed: Underground 2
8,9,Nier Automata
9,10,Legend Of Zelda: Majora's Mask


Let's strip the white space from the headers again. 

In [269]:
mp3_downloads = mp3_downloads.rename(columns=lambda x: x.strip())
mp3_downloads.columns

Index(['#', 'Album'], dtype='object')

In [270]:
vgmdb_least_pop_album = vgmdb_least_pop_album.loc[:, ['Rank', 'Album Name', 
                                                      'Rating', 'Popularity']]
vgmdb_least_pop_album = vgmdb_least_pop_album.rename(
    columns={'Rank': 'VGMDB Soundtrack Rank', 'Album Name': 'Game', 
             'Rating': 'VGMDB Soundtrack Rating', 
             'Popularity': 'VGMDB Soundtrack Popularity'})
vgmdb_least_pop_album.head(20)

,VGMDB Soundtrack Rank,Game,VGMDB Soundtrack Rating,VGMDB Soundtrack Popularity
0,1,Diabolik Lovers,3.80,0.0006
1,2,Eiga Precure Dream Stars!,4.10,0.0006
2,3,Diabolik Lovers,4.04,0.0006
3,4,Haikyu,4.04,0.0006
4,5,Eiga Kirakira precure a La Mode Paris-tto! Omo...,4.50,0.0006
5,6,Nightmare,4.31,0.0006
6,14,Dive & Drama,5.00,0.0006
7,16,Square Enix,4.58,0.0029
8,18,Kirakira precure a La Mode 1: Precure sound de...,4.33,0.0040
9,19,Ao Haru Ride,4.29,0.0043


In [271]:
mp3_downloads = mp3_downloads.rename(columns={'#': 'mp3 Downloads Rank', 
                                             'Album': 'Game'})
mp3_downloads.head(20)

,mp3 Downloads Rank,Game
0,1,Persona 5
1,2,Need For Speed: Most Wanted
2,3,Super Mario World
3,4,Minecraft
4,5,Legend Of Zelda: Ocarina Of Time
5,6,Super Smash Bros Brawl: Gamerip
6,7,Persona 4
7,8,Need For Speed: Underground 2
8,9,Nier Automata
9,10,Legend Of Zelda: Majora's Mask


In [272]:
mp3_merge_vgmdb = pd.merge(vgmdb_least_pop_album, mp3_downloads, how='outer', on='Game')
mp3_merge_vgmdb

,VGMDB Soundtrack Rank,Game,VGMDB Soundtrack Rating,VGMDB Soundtrack Popularity,mp3 Downloads Rank
0,1.0,Diabolik Lovers,3.80,0.0006,NaN
1,23.0,Diabolik Lovers,4.10,0.0077,NaN
2,2.0,Eiga Precure Dream Stars!,4.10,0.0006,NaN
3,3.0,Diabolik Lovers,4.04,0.0006,NaN
4,4.0,Haikyu,4.04,0.0006,NaN
5,5.0,Eiga Kirakira precure a La Mode Paris-tto! Omo...,4.50,0.0006,NaN
6,6.0,Nightmare,4.31,0.0006,NaN
7,14.0,Dive & Drama,5.00,0.0006,NaN
8,16.0,Square Enix,4.58,0.0029,NaN
9,379.0,Square Enix,3.17,0.0749,NaN


This will need to be looked into more with regards to data cleaning, as there are only 17 overlapping entries. For now, I'm keeping it as an outer join until I look into it. 

In [273]:
final_dataset = pd.merge(vgmdb_wiki_kaggle, mp3_merge_vgmdb, how='inner', on='Game')
final_dataset

,Game,Platform,Year_of_Release,Genre,Publisher,Critic_Score,User_Score,Developer,ESRB Rating,Composer,Composer Birthday,Composer Rank,Composer Rating,VGMDB Soundtrack Rank,VGMDB Soundtrack Rating,VGMDB Soundtrack Popularity,mp3 Downloads Rank
0,Wii Sports,Wii,2006,Sports,Nintendo,76.0,8,Nintendo,E,Kazumi Totaka,1967-08-23,6777.0,4.07,NaN,NaN,NaN,278.0
1,Wii Sports,Wii,2006,Sports,Nintendo,76.0,8,Nintendo,E,Kazumi Totaka,1967-08-23,6777.0,4.07,NaN,NaN,NaN,278.0
2,Pokemon Red/green/blue/yellow,GB,1996,Role-Playing,Nintendo,NaN,NaN,NaN,NaN,Junichi Masuda,1968-01-12,5712.0,4.14,NaN,NaN,NaN,235.0
3,Pokemon Red/green/blue/yellow,GB,1996,Role-Playing,Nintendo,NaN,NaN,NaN,NaN,Junichi Masuda,1968-01-12,5712.0,4.14,NaN,NaN,NaN,948.0
4,Tetris,GB,1989,Puzzle,Nintendo,NaN,NaN,NaN,NaN,Hirokazu Tanaka,1957-12-13,6489.0,4.09,NaN,NaN,NaN,364.0
5,Tetris,GB,1989,Puzzle,Nintendo,NaN,NaN,NaN,NaN,Jun Senoue,1970-08-02,6091.0,4.12,NaN,NaN,NaN,364.0
6,Tetris,GB,1989,Puzzle,Nintendo,NaN,NaN,NaN,NaN,Kazumi Totaka,1967-08-23,6777.0,4.07,NaN,NaN,NaN,364.0
7,Tetris,GB,1989,Puzzle,Nintendo,NaN,NaN,NaN,NaN,Tomoya Ohtani,1974-07-01,5079.0,4.18,NaN,NaN,NaN,364.0
8,Tetris,NES,1988,Puzzle,Nintendo,NaN,NaN,NaN,NaN,Hirokazu Tanaka,1957-12-13,6489.0,4.09,NaN,NaN,NaN,364.0
9,Tetris,NES,1988,Puzzle,Nintendo,NaN,NaN,NaN,NaN,Jun Senoue,1970-08-02,6091.0,4.12,NaN,NaN,NaN,364.0


Final dataset above, but again I will need to go back and look into data cleaning to ensure more album names match up with each other when doing an inner join, because the current number is...low

In [274]:
final_dataset = final_dataset.drop_duplicates()
final_dataset

,Game,Platform,Year_of_Release,Genre,Publisher,Critic_Score,User_Score,Developer,ESRB Rating,Composer,Composer Birthday,Composer Rank,Composer Rating,VGMDB Soundtrack Rank,VGMDB Soundtrack Rating,VGMDB Soundtrack Popularity,mp3 Downloads Rank
0,Wii Sports,Wii,2006,Sports,Nintendo,76.0,8,Nintendo,E,Kazumi Totaka,1967-08-23,6777.0,4.07,NaN,NaN,NaN,278.0
2,Pokemon Red/green/blue/yellow,GB,1996,Role-Playing,Nintendo,NaN,NaN,NaN,NaN,Junichi Masuda,1968-01-12,5712.0,4.14,NaN,NaN,NaN,235.0
3,Pokemon Red/green/blue/yellow,GB,1996,Role-Playing,Nintendo,NaN,NaN,NaN,NaN,Junichi Masuda,1968-01-12,5712.0,4.14,NaN,NaN,NaN,948.0
4,Tetris,GB,1989,Puzzle,Nintendo,NaN,NaN,NaN,NaN,Hirokazu Tanaka,1957-12-13,6489.0,4.09,NaN,NaN,NaN,364.0
5,Tetris,GB,1989,Puzzle,Nintendo,NaN,NaN,NaN,NaN,Jun Senoue,1970-08-02,6091.0,4.12,NaN,NaN,NaN,364.0
6,Tetris,GB,1989,Puzzle,Nintendo,NaN,NaN,NaN,NaN,Kazumi Totaka,1967-08-23,6777.0,4.07,NaN,NaN,NaN,364.0
7,Tetris,GB,1989,Puzzle,Nintendo,NaN,NaN,NaN,NaN,Tomoya Ohtani,1974-07-01,5079.0,4.18,NaN,NaN,NaN,364.0
8,Tetris,NES,1988,Puzzle,Nintendo,NaN,NaN,NaN,NaN,Hirokazu Tanaka,1957-12-13,6489.0,4.09,NaN,NaN,NaN,364.0
9,Tetris,NES,1988,Puzzle,Nintendo,NaN,NaN,NaN,NaN,Jun Senoue,1970-08-02,6091.0,4.12,NaN,NaN,NaN,364.0
10,Tetris,NES,1988,Puzzle,Nintendo,NaN,NaN,NaN,NaN,Kazumi Totaka,1967-08-23,6777.0,4.07,NaN,NaN,NaN,364.0


In [275]:
from IPython.display import HTML
import base64 

In [276]:
def create_download_link( df, title = "Download CSV file", filename = "johnson_eric_final_dataset.csv"):  
    csv = df.to_csv()
    b64 = base64.b64encode(csv.encode())
    payload = b64.decode()
    html = '<a download="{filename}" href="data:text/csv;base64,{payload}" target="_blank">{title}</a>'
    html = html.format(payload=payload,title=title,filename=filename)
    return HTML(html)

create_download_link(final_dataset)

The code to download this csv file came from:
https://blog.softhints.com/jupyter-ipython-download-files/#zip